In [1]:
import torch
import numpy as np
import pandas  as pd
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import os
import kagglehub

In [2]:
device="cuda"

In [3]:
path=kagglehub.dataset_download("zalando-research/fashionmnist")

100%|██████████| 68.8M/68.8M [00:00<00:00, 224MB/s]

Extracting files...


In [4]:
os.listdir(path)

['fashion-mnist_test.csv',
 'train-images-idx3-ubyte',
 't10k-labels-idx1-ubyte',
 'train-labels-idx1-ubyte',
 't10k-images-idx3-ubyte',
 'fashion-mnist_train.csv']

In [5]:
train_df=pd.read_csv(path + "/fashion-mnist_train.csv")
test_df=pd.read_csv(path + "/fashion-mnist_test.csv")
device= "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
x_train=train_df.drop("label",axis=1)
y_train=train_df["label"]
x_test=test_df.drop("label",axis=1)
y_test=test_df["label"]


In [7]:
from torchvision.transforms import transforms

customtransform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]

)

In [20]:

class CustomDataset(Dataset):
    def __init__(self,x,y,transform):
        self.x=x.values
        self.y=y.values
        self.transform=transform

    def __len__(self):
        return len(self.x)
    def __getitem__(self,index):
        image=self.x[index].reshape(28,28)

        image=image.astype(np.uint8)

        image=np.stack([image]*3,axis=-1)

        image=Image.fromarray(image)

        image=self.transform(image)
        return image,torch.tensor(self.y[index],dtype=torch.long)

train_dataset=CustomDataset(x_train,y_train,transform=customtransform)
test_dataset=CustomDataset(x_test,y_test,transform=customtransform)
train_load=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_load=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [21]:
import torchvision.models as models

In [ ]:
# models.list_models()

In [23]:
vgg16=models.vgg16(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
for params in vgg16.parameters():
    params.requires_grad=False

In [25]:
vgg16.classifier=nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

In [26]:
vgg16=vgg16.to(device)


In [27]:
learning_rate=0.001
epochs=5

In [28]:

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(vgg16.classifier.parameters(),lr=learning_rate)

In [34]:
for epoch in range(epochs):

    total_loss=0

    for x,y in train_load:

        x,y=x.to(device),y.to(device)

        out=vgg16(x)

        loss=criterion(out,y)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        total_loss+=loss.item()

    print(total_loss/len(train_load))


0.11326574257283938
0.08483118599206985
0.06640004782968074
0.05664987199882207
0.04688854813019098


In [35]:
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [36]:
#test_dataset
total=0
correct=0
with torch.no_grad():
    for x,y in test_load:
        x,y=x.to(device),y.to(device)
        out=vgg16(x)
        _,predicted=torch.max(out,1)
        total+=x.shape[0]
        correct+=(predicted==y).sum().item()
    print(correct/total)

0.924


In [37]:
# train_dataset
total=0
correct=0
with torch.no_grad():
    for x,y in train_load:
        x,y=x.to(device),y.to(device)
        out=vgg16(x)
        _,predicted=torch.max(out,1)
        total+=x.shape[0]
        correct+=(predicted==y).sum().item()
    print(correct/total)

0.98735
